## Running Meta Llama 3.1 on Google Colab using Hugging Face transformers library
This notebook goes over how you can set up and run Llama 3.1 using Hugging Face transformers library
<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/quickstart/Running_Llama2_Anywhere/Running_Llama_on_HF_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Steps at a glance:
This demo showcases how to run the example with already converted Llama 3.1 weights on [Hugging Face](https://huggingface.co/meta-llama). Please Note: To use the downloads on Hugging Face, you must first request a download as shown in the steps below making sure that you are using the same email address as your Hugging Face account.

To use already converted weights, start here:
1. Request download of model weights from the Llama website
2. Login to Hugging Face from your terminal using the same email address as (1). Follow the instructions [here](https://huggingface.co/docs/huggingface_hub/en/quick-start). 
3. Run the example


Else, if you'd like to download the models locally and convert them to the HF format, follow the steps below to convert the weights:
1. Request download of model weights from the Llama website
2. Clone the llama repo and get the weights
3. Convert the model weights
4. Prepare the script
5. Run the example

### Using already converted weights

#### 1. Request download of model weights from the Llama website
Request download of model weights from the Llama website
Before you can run the model locally, you will need to get the model weights. To get the model weights, visit the [Llama website](https://llama.meta.com/) and click on “download models”. 

Fill  the required information, select the models “Meta Llama 3.1” and accept the terms & conditions. You will receive a URL in your email in a short time.

#### 2. Prepare the script

We will install the Transformers library and Accelerate library for our demo.

The `Transformers` library provides many models to perform tasks on texts such as classification, question answering, text generation, etc.
The `accelerate` library enables the same PyTorch code to be run across any distributed configuration of GPUs and CPUs.


In [1]:
!pip install transformers
!pip install accelerate

Next, we will import AutoTokenizer, which is a class from the transformers library that automatically chooses the correct tokenizer for a given pre-trained model, import transformers library and torch for PyTorch.


In [2]:
from transformers import AutoTokenizer
import transformers
import torch

Then, we will set the model variable to a specific model we’d like to use. In this demo, we will use the 8b chat model `meta-llama/Meta-Llama-3.1-8B-Instruct`. Using Meta models from Hugging Face requires you to

1. Accept Terms of Service for Meta Llama 3.1 on Meta [website](https://llama.meta.com/llama-downloads).
2. Use the same email address from Step (1) to login into Hugging Face.

Follow the instructions on this Hugging Face page to login from your [terminal](https://huggingface.co/docs/huggingface_hub/en/quick-start). 

In [3]:
pip install --upgrade huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [4]:
from huggingface_hub import login
login()

In [5]:
# import os
# os.environ['http_proxy'] = 'http://172.25.80.1:7890'  # 替换为你的代理地址
# os.environ['https_proxy'] = 'http://172.25.80.1:7890'
# os.getenv('https_proxy')

In [6]:
# model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model)


Now, we will use the `from_pretrained` method of `AutoTokenizer` to create a tokenizer. This will download and cache the pre-trained tokenizer and return an instance of the appropriate tokenizer class.


In [7]:
pipeline = transformers.pipeline(
"text-generation",
      model=model,
      torch_dtype=torch.float16,
 device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


#### 3. Run the example

Now, let’s create the pipeline for text generation. We’ll also set the device_map argument to `auto`, which means the pipeline will automatically use a GPU if one is available.

Let’s also generate a text sequence based on the input that we provide. 

In [8]:
sequences = pipeline(
    'I have tomatoes, basil and cheese at home. What can I cook for dinner?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation = True,
    max_length=400,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Result: I have tomatoes, basil and cheese at home. What can I cook for dinner?
You can make a classic Caprese salad, a simple yet delicious dish that combines the flavors of Italy. Here's a quick recipe:

Ingredients:

* 3 large tomatoes, sliced into 1/4-inch thick rounds
* 1/4 cup fresh basil leaves, chopped
* 8 oz (225g) mozzarella cheese, sliced into 1/4-inch thick rounds
* Salt and pepper to taste
* Olive oil for drizzling (optional)

Instructions:

1. Arrange the tomato slices on a large plate or platter.
2. Top each tomato slice with a round of mozzarella cheese.
3. Sprinkle chopped basil leaves over the cheese.
4. Season with salt and pepper to taste.
5. Drizzle with olive oil, if desired, for a bit of extra flavor.
6. Serve immediately and enjoy!

Alternatively, you can also make a Caprese-style pizza by topping a pre-made pizza crust or a baguette slice with the tomato, mozzarella, and basil, and baking it in the oven until the cheese is melted and bubbly.

Which option sounds

<br>

### Downloading and converting weights to Hugging Face format

#### 1. Request download of model weights from the Llama website
Request download of model weights from the Llama website
Before you can run the model locally, you will need to get the model weights. To get the model weights, visit the [Llama website](https://llama.meta.com/) and click on “download models”. 

Fill  the required information, select the models "Meta Llama 3" and accept the terms & conditions. You will receive a URL in your email in a short time.

#### 2. Clone the llama repo and get the weights
Git clone the [Meta Llama 3 repo](https://github.com/meta-llama/llama3). Run the `download.sh` script and follow the instructions. This will download the model checkpoints and tokenizer.

This example demonstrates a Meta Llama 3.1 model with 8B-instruct parameters, but the steps we follow would be similar for other llama models, as well as for other parameter models.

#### 3. Convert the model weights using Hugging Face transformer from source

* `python3 -m venv hf-convertor`
* `source hf-convertor/bin/activate`
* `git clone https://github.com/huggingface/transformers.git`
* `cd transformers`
* `pip install -e .`
* `pip install torch tiktoken blobfile accelerate`
* `python3 src/transformers/models/llama/convert_llama_weights_to_hf.py --input_dir ${path_to_meta_downloaded_model} --output_dir ${path_to_save_converted_hf_model} --model_size 8B --llama_version 3.1`


#### 4. Prepare the script
Import the following necessary modules in your script: 
* `AutoModel` is the Llama 3 model class
* `AutoTokenizer` prepares your prompt for the model to process
* `pipeline` is an abstraction to generate model outputs

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = "${path_the_converted_hf_model}"
model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        device_map="auto",
    )
tokenizer = AutoTokenizer.from_pretrained(model_dir)


We need a way to use our model for inference. Pipeline allows us to specify which type of task the pipeline needs to run (`text-generation`), specify the model that the pipeline should use to make predictions (`model`), define the precision to use this model (`torch.float16`), device on which the pipeline should run (`device_map`)  among various other options. 


In [2]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Now we have our pipeline defined, and we need to provide some text prompts as inputs to our pipeline to use when it runs to generate responses (`sequences`). The pipeline shown in the example below sets `do_sample` to True, which allows us to specify the decoding strategy we’d like to use to select the next token from the probability distribution over the entire vocabulary. In our example, we are using top_k sampling. 

By changing `max_length`, you can specify how long you’d like the generated response to be. 
Setting the `num_return_sequences` parameter to greater than one will let you generate more than one output.

In your script, add the following to provide input, and information on how to run the pipeline:


#### 5. Run the example

In [ ]:
sequences = pipeline(
    'I have tomatoes, basil and cheese at home. What can I cook for dinner?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
)
for seq in sequences:
    print(f"{seq['generated_text']}")
